# Imports

In [1]:
import os 
os.chdir("/home/ivanr/git/document_information_extraction/")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
from src.data.wikipedia.wiki_data_base import retrieve_query, retrive_observations_from_ids

ImportError: cannot import name 'retrive_observations_from_ids' from 'src.data.wikipedia.wiki_data_base' (/home/ivanr/git/document_information_extraction/src/data/wikipedia/wiki_data_base.py)

# Statics

In [ ]:
from src.data.data_statics import INTERIM_DATA_PATH

# Read data

In [ ]:
db_path = INTERIM_DATA_PATH / "wiki_db_dumps_test_temp.db"
query = """
    SELECT *
    FROM summary_similarity
   -- limit 125000
"""

In [ ]:
a = set(
    [
        i
        for i in string.ascii_letters
        + string.ascii_lowercase
        + string.ascii_uppercase
        + string.punctuation
        + string.digits
        + string.printable
        + "?"
        + "!"
    ]
)
a.difference_update({"[", "]", "\\", '"', "#", "^", "_", "`", "{", "|", "}", "~"})

In [ ]:
a

In [ ]:
data = retrieve_query(query,db_path)

# Clean up

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
    
import string

def isEnglish(s):
    return s.translate( string.punctuation).isalnum()

In [ ]:
isEnglish("சிவப்பிரகாசர்")

In [ ]:
# for sent in data:
#     if isEnglish(sent[-1]):
#         pass
#     else:
#         print(sent[-1])
#         print(" ")
        

In [ ]:
df = pd.DataFrame(
    [[sent[i] for i in range(6)] for sent in data],
    columns=["id", "page_id", "sent_num", "precision", "recall", "f1"],
)
df

# Plot distributions

## Single variable distribution

In [ ]:
for var in ["precision", "recall", "f1"]:
    plt.figure(figsize = (12.5,7.5))
    plt.title(var, fontsize = 20)
    quantile = df[var].quantile(.99)
    df[var].hist(bins = 100, density = True)
    plt.show()

## Joint distributions

# Sample evaluation

In [ ]:
def evaluate_threshold(df, threshold, metric,n_samples = 5):
    overview = (
        df[df[metric] >= threshold]
        .sort_values(by=metric, ascending=True)
        .head(n_samples)
        .reset_index()
    )
    display(overview)
    overview = list(overview["id"])

    for i in overview:
        print("\n", "-" * 20, i, "-" * 20)
        print("Summary\n")
        print(data_dict[i][-1])
        print("\ntext\n")
        print(
            (
                retrive_observations_from_ids(
                    ids=[int(i.split("_")[0])], out_f=str(db_path)
                )[0][1]
            ).decode()
        )

## f1

In [ ]:
for q in np.quantile(df["f1"], [(i+.5)/10 for i in range(10)]):
    evaluate_threshold(df,.5, "f1")

## Precision 

In [ ]:
for q in np.quantile(df["precision"], [(i+.5)/10 for i in range(10)]):
    evaluate_threshold(df,.5, "precision")

## Recall

In [ ]:
for q in np.quantile(df["recall"], [(i+.5)/10 for i in range(10)]):
    evaluate_threshold(df,.5, "recall")